Data Analysis:
- Your working directory must contain the python file (LMRT-NNanotech-2023.py), and excel data file (template.xlsx). The output folders (Kinetics, FC, Rates) will be saved in this same directory.

Raw_template.xlsx:
- It is important that you use the template (Raw_template.xlsx) to load your raw data. Simply replace the data in the first excel sheet (named "Raw") and make sure the data is correctly populated in the "*-kinetics" and "*-FC" sheets.
- In the given template, data from two screens are loaded (Tissue 1 and Tissue 2 are tested with 42 different fluorescent peptides in duplicates, one fluorescence read is recorded every 3 minutes during 2 hours). You can add other sheets to the template. Make sure they are named with the suffix "kinetics" and "FC" before you run the python analysis. 

Data Visualization:
- If you want to automate the visualization of processed data with PRISM, make sure you have the PRISM-Script.pzc and PRISM-Template.pzfx in your working directory.
- If you are on mac OS, you can automate launching PRISM using the PRISM-Apple-Launcher.scpt, which is called in the last line of the LMRT-NNanotech-2023.py. If you are running another operating system, comment out the last line of LMRT-NNanotech-2023.py, and manually open PRISM-Script.pzc (which should launch PRISM).

Now, we will go through the python code and explain section by section: 

Star by importing the necessary packages:

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import os
from itertools import islice

In [2]:
#Create directory to house all outputs
folder_name=input("What would you like to name the folder? ")
os.mkdir('./'+(folder_name)+'')

Import and convert the raw excel (template.xlsx) data file (must be in the same directory as the python file, or adjust the path to wherever your data file is) into csv files: 
- This section will result in two new directories: 
1) "Kinetics" will contain a separate csv file with kinetics data for each different tissue.
2) "FC" will contain a separate csv file with the fold change data for each different tissue.

In [3]:
pd.options.mode.chained_assignment = None
#Convert Excel Sheets (Kinetics and Fold Change) into individual CSV Files for each Tissue Sample

os.mkdir('./Kinetics')
excel_file=input("Enter name of Excel File ")
f=input("index of first Kinetics sheet ") #in the template given, this would be "3"
l=input("index of last Kinetics sheet ") #in the template given, this would be "4"
sheets_dict=pd.read_excel(excel_file, sheet_name=None)
for name,sheet in islice(sheets_dict.items(),int(f),int(l)+1):
    sheet.to_csv(os.path.join('./Kinetics', name + '.csv'), index= None, header =False)

os.mkdir('./FC')
f=input("index of first FC sheet ") #in the template given, this would be "5"
l=input("index of last FC sheet ")  #in the template given, this would be "6"
for name,sheet in islice(sheets_dict.items(),int(f),int(l)+1):
    sheet.to_csv(os.path.join('./FC', name + '.csv'), index= None, header =False)

- This section will result in a directory called "Rates" containing:
1) a separate csv file with kinetic cleavage rates of each tissue
2) a csv file combining the kinetic cleavage rates of all tissues

In [4]:
#Calculating Linear Regression slope (cleavage rate) for each tissue
os.mkdir('./Rate')

a = input("Do you want to use the same number of time points for all tissues? (y/n) ") #in the template given, this would be "y"
if a == "y":
    t=input("How many time points are you considering? ") #in the template given, this would be "8" since the linear range lasts for approximately the first 8 time points for both tissues

for filename in os.scandir('./Kinetics'):
    if(filename.path.endswith("csv")):
        name = os.path.splitext(os.path.basename(filename))[0]
        data=pd.read_csv(filename, header=None)
        av=np.zeros(shape=(len(data),int((len(data.columns)+1)/2)), dtype=object)

        j=0
        for i in range(int((len(data.columns)+1)/2)):
            av[0][i]=data[j][0]
            j=j+2
        for i in range(len(data)):
            av[i][0]=data[0][i]


        for i in range(1, len(av) ):
            k=1
            for j in range(1, len(av[0])):
                av[i][j]=(pd.to_numeric(data[k][i]) + (pd.to_numeric(data[k+1][i])))/2
                k=k+2
        
        average = pd.DataFrame(av)
        
        header=average.iloc[0]
        average=average[1:]
        average.columns=header
        average = average.apply(pd.to_numeric)
        
        #ask for number of time points
        if a == "n":
            t=input("How many time points are you considering for " + name + "? ")
            average=average[:int(t)]
        else:
            average=average[:int(t)]


        x=np.array(average['Time (min)']).reshape(-1,1)
        rate=average.iloc[:1, :]
        
        rate.iloc[0,0] = name
        rate.columns.values[0]= t +' Time Points'

        for i in range(1,average.shape[1]):
            y=np.array(average.iloc[:,i]).reshape(-1,1)
            regressor=LinearRegression()
            regressor.fit(x,y)
            rate.iloc[0,i]=float(regressor.coef_)
            
        rate.to_csv(os.path.join('./Rate/', name + '_Rate.csv'))


#Combining Cleavage Rates for all tissues in one file
import glob
os.chdir('./Rate/')
extension='csv'
all_filenames=[i for i in glob.glob('*.{}'.format(extension))]
combined_csv=pd.concat([pd.read_csv(f, index_col=0) for f in all_filenames])
combined_csv.to_csv("Rates.csv", index=False)

#Converting the Cleavage rate into PRISM friendly format
data=pd.read_csv("Rates.csv",  index_col=0, header=None)
data=data.transpose()
data.to_csv('Rates.csv', index=False)

All resulting csv files are PRISM compatible and can seamlessly be imported into PRISM. If you want to automate PRISM visualization you can either:
1) Run the PRISM-Script.pzc (this script will use PRISM-Template.pzfx as a template). Open the PRISM-Script.pzc with a text editor or PRISM to edit the name of the directory (it should be the same directory where the output of the python script and the PRISM-Template.pzfx are) in every line that starts with "Set Path".
2) If you are working on Mac OS, the last section of the python script will automatically launch PRISM using the PRISM-Apple_Launcher.scpt which must be in the same directory as the python outputs and PRISM-Template.pzfx. If you are using another operating system, comment out this line.

In [5]:
os.mkdir('../PRISM')
os.system('cd ..; osascript PRISM-Apple-Launcher.scpt; mv Kinetics Rate PRISM FC '+(folder_name)+'')


0